# Исследовательский анализ данных, предобработка и обогащение данных

## 0 - Настройка

In [1]:
import numpy as np
import pandas as pd
import os
import requests
import json

In [2]:
path_download = 'D:/000_Learning/2021_TSU/_PersonalFolderWithProject/vac_info'
path_save = 'D:/000_Learning/2021_TSU/_PersonalFolderWithProject/vac_EDA'

In [3]:
df_general = pd.read_csv(os.path.join(path_download, 'vac_general.csv'))
df_descr = pd.read_csv(os.path.join(path_download, 'vac_descr.csv'))
df_key_skills = pd.read_csv(os.path.join(path_download, 'vac_key_skills.csv'))
df_role = pd.read_csv(os.path.join(path_download, 'vac_role.csv'))
df_spec = pd.read_csv(os.path.join(path_download, 'vac_spec.csv'))

In [4]:
# уберем поле с индексом
df_general = df_general.drop(columns=['Unnamed: 0'])
df_role = df_role.drop(columns=['Unnamed: 0'])
df_spec = df_spec.drop(columns=['Unnamed: 0'])
df_key_skills = df_key_skills.drop(columns=['Unnamed: 0'])
df_descr = df_descr.drop(columns=['Unnamed: 0'])

## 1 df_general

In [5]:
df_general.head(2)

,id,name,area_name,salary_from,salary_to,salary_ccy,salary_gross,type_name,address_lat,address_lng,experience_name,schedule_name,employment_name,employer_id,employer_name,published_at,alternate_url
0,14694668,Junior разработчик (стажер),Тула,20000.0,NaN,RUR,0.0,Открытая,54.185099,37.612012,Нет опыта,Полный день,Полная занятость,1196086.0,ЮниВеб,2021-10-31T19:51:34+0300,https://hh.ru/vacancy/14694668
1,14820007,Программист 1С,Тула,70000.0,NaN,RUR,1.0,Открытая,54.182090,37.573411,От 1 года до 3 лет,Полный день,Полная занятость,200522.0,"СофтЭксперт, г. Тула",2021-11-02T18:13:15+0300,https://hh.ru/vacancy/14820007


In [6]:
df_general.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87306 entries, 0 to 87305
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               87306 non-null  int64  
 1   name             87306 non-null  object 
 2   area_name        87306 non-null  object 
 3   salary_from      42263 non-null  float64
 4   salary_to        24375 non-null  float64
 5   salary_ccy       46648 non-null  object 
 6   salary_gross     46609 non-null  object 
 7   type_name        87306 non-null  object 
 8   address_lat      37965 non-null  float64
 9   address_lng      37965 non-null  float64
 10  experience_name  87306 non-null  object 
 11  schedule_name    87306 non-null  object 
 12  employment_name  87306 non-null  object 
 13  employer_id      87245 non-null  float64
 14  employer_name    87306 non-null  object 
 15  published_at     87306 non-null  object 
 16  alternate_url    87306 non-null  object 
dtypes: float64(5

In [7]:
df_general.isnull().mean()

id                 0.000000
name               0.000000
area_name          0.000000
salary_from        0.515921
salary_to          0.720810
salary_ccy         0.465695
salary_gross       0.466142
type_name          0.000000
address_lat        0.565150
address_lng        0.565150
experience_name    0.000000
schedule_name      0.000000
employment_name    0.000000
employer_id        0.000699
employer_name      0.000000
published_at       0.000000
alternate_url      0.000000
dtype: float64

In [8]:
# необходимо изменить тип данных у поля employer_id на int
df_general.loc[df_general['employer_id'].isnull(), 'employer_id'] = 0
df_general['employer_id'] = df_general['employer_id'].astype('int32')
# необходимо изменить тип данных у поля published_at на datetime
df_general['published_at'] = pd.to_datetime(df_general['published_at'])

In [10]:
print('Вакансии с {} по {}'.format(df_general['published_at'].min().date(), df_general['published_at'].max().date()))

Вакансии с 2021-10-07 по 2021-11-09


#### Промежуточные выводы:
- Посреством API hh.ru выкачали вакансии со специализацией "Информационные технологии, интернет, телеком" (https://hh.ru/catalog/informacionnye-tehnologii-internet-telekom) за период с 2021-10-07 по 2021-11-09.
- Общее число вакансий: 87 306 вакансий

## 2 Роли и специализации

#### df_role

In [11]:
df_role.head(2)

,id,prof_role
0,14694668,"Программист, разработчик"
1,14820007,"Программист, разработчик"


In [12]:
df_role.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87306 entries, 0 to 87305
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         87306 non-null  int64 
 1   prof_role  87306 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.3+ MB


In [13]:
df_role.groupby(['id'])['prof_role'].count().sort_values(ascending=False)

id
6555939     1
48923125    1
48923255    1
48923244    1
48923243    1
           ..
48153064    1
48153003    1
48152950    1
48152635    1
49390560    1
Name: prof_role, Length: 87306, dtype: int64

#### df_spec

In [14]:
df_spec.head(2)

,id,specialization
0,14694668,"Начальный уровень, Мало опыта"
1,14694668,"Программирование, Разработка"


In [15]:
df_spec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302765 entries, 0 to 302764
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   id              302765 non-null  int64 
 1   specialization  302765 non-null  object
dtypes: int64(1), object(1)
memory usage: 4.6+ MB


In [16]:
df_spec.groupby(['id'])['specialization'].count().sort_values(ascending=False)

id
6555939     6
48529262    6
49229328    6
48528694    6
48528821    6
           ..
48244257    1
49127270    1
48244288    1
48244395    1
49390560    1
Name: specialization, Length: 87306, dtype: int64

#### Объединяем датафреймы

In [17]:
df = df_general.merge(df_role, how='left', on='id', right_index=False)
df_temp = df.merge(df_spec, how='left', on='id', right_index=False)

#### Обрабатываем вакансии - кластеризация по профессиям через регулярки

In [18]:
# Получим список профессиональных ролей в ИТ согласно https://api.hh.ru/professional_roles
url = 'https://api.hh.ru/professional_roles'
req = requests.get(url)
data = req.content.decode()
req.close()
roles_json = json.loads(data)
it_roles = roles_json['categories'][7]['roles']
df_it_roles = pd.json_normalize(it_roles)
df_it_roles['IT_ProRole'] = True
df_it_roles['name'].unique()

array(['Аналитик', 'Арт-директор, креативный директор', 'Гейм-дизайнер',
       'Дизайнер, художник',
       'Директор по информационным технологиям (CIO)',
       'Менеджер продукта', 'Программист, разработчик',
       'Руководитель группы разработки', 'Руководитель проектов',
       'Сетевой инженер', 'Системный администратор', 'Системный инженер',
       'Специалист по информационной безопасности',
       'Специалист технической поддержки', 'Тестировщик',
       'Технический директор (CTO)', 'Технический писатель'], dtype=object)

In [19]:
def my_filter(row):
    
    game_prod_web_da = [
        'game analyst',
        'game data analyst',
        'growth hacker',
        'product analyst',
        'product data analyst',
        'web - аналитик',
        'web – аналитик',
        'web analyst / вэб-аналитик',
        'web аналитик',
        'web/app аналитик',
        'web-аналитик',
        'аналитик (unit экономика)',
        'аналитик по продуктовой отчётности',
        'аналитик по продукту',
        'аналитик продукта',
        'аналитик продукта / данных',
        'аналитик-исследователь (my.games)',
        'аналитик-маркетолог(olap, bi)',
        'аналитик-математик',
        'веб - аналитик',
        'веб аналитик',
        'веб-аналитик',
        'ведущий аналитик (gamedev)',
        'вэб-аналитик',
        'маркетинговый аналитик',
        'продуктовый аналитик',
        'специалист по сквозной аналитике',
        'аналитик / ведущий аналитик (mobile)',
        'продуктовый/bi-аналитик',

    ]
    
    data_analyst = [
        'big data аналитик',
        'bigdata analyst',
        'bigdata аналитик',
        'data analysis (da)',
        'data analyst',
        'data аналитик',
        'data аналитик | bigdata',
        'data аналитик/аналитик данных',
        'data-аналитик',
        'junior analyst, moscow',
        'sales analyst',
        'sql аналитик',
        'sql-аналитик',
        'sql-специалист',
        'аналитик (big data)',
        'аналитик (sql + анализ данных)',
        'аналитик (sql)',
        'аналитик (sql, bi)',
        'аналитик (sql, oracle)',
        'аналитик (анализ бэк маржи)',
        'аналитик (анализ и планирование промо)',
        'аналитик (бд)',
        'аналитик big data (почта)',
        'аналитик dwh',
        'аналитик dwh (bi)',
        'аналитик etl',
        'аналитик python',
        'аналитик rpa',
        'аналитик sas',
        'аналитик sberx',
        'аналитик sql',
        'аналитик sql (big data)',
        'аналитик terrasoft',
        'аналитик базы данных (бд)',
        'аналитик данных',
        'аналитик данных big data (sql)',
        'аналитик отдела маркетинга / аналитик данных',
        'аналитик отдела продаж',
        'аналитик по ассортименту',
        'аналитик по количественным расчетам (quantitative analyst)',
        'аналитик по обработке данных',
        'аналитик по продажам',
        'аналитик по продажам (power bi)',
        'аналитик по промо',
        'аналитик по работе с данными',
        'аналитик по работе с клиентской базой',
        'аналитик по работе с маркетплейсами',
        'аналитик по сегментации',
        'аналитик по управлению данными (аналитик проекта)',
        'аналитик по управлению изменениями (data analyst)',
        'аналитик по ценообразованию',
        'аналитик по эффективности',
        'аналитик продаж',
        'дата аналитик',
        'дата-аналитик',
        'дата-аналитик/старший аналитик',
        'менеджер по анализу больших данных (big data)',
        'специалист по анализу данных',
        'специалист по аналитике',
        'специалист со знанием sql',
        'специалист-аналитик',
        'data analytics',
        'data processing senior specialist / старший специалист по обработке данных',
        'data analyst',
        'data-analyst',
        'специалист по обработке данных со знанием spss / data processing specialist',
        'сборщик данных data set',
        'консультант в области обработки, анализа и визуализации данных',
        'аналитик базы данных msql',
        'аналитик модели данных',
        'специалист по вовлеченности клиентов и анализу данных',
        'старший исследователь больших данных',
        'эксперт по анализу данных',
        'аналитик больших данных'
    ]
    
    bi_analyst = [
        'bi analyst',
        'bi specialist',
        'bi technologist',
        'bi аналитик',
        'bi-специалист',
        'power bi специалист',
        'product/bi analyst',
        'reporting analyst (power bi)',
        'reporting specialist / аналитик',
        'аналитик ( отчетность )',
        'аналитик (bi систем)',
        'аналитик (bi)',
        'аналитик (power bi)',
        'аналитик (аналитическая поддержка и развитие систем отчетности)',
        'аналитик (отчетность)',
        'аналитик bi (tableau)',
        'аналитик bi (стажер-аналитик)',
        'аналитик power bi',
        'аналитик tableau',
        'аналитик данных bi',
        'аналитик отдела сопровождения bi решений',
        'аналитик по развитию отчетности',
        'аналитик по разработке bi',
        'аналитик/bi аналитик',
        'бизнес аналитик bi системы',
        'бизнес аналитик power bi',
        'бизнес-аналитик в power bi',
        'консультант bi (tableau)',
        'консультант bi / ml',
        'консультант qlik',
        'консультант qlikview',
        'специалист по бизнес-отчетности на базе power bi',
        'технический аналитик power bi',
        'эксперт power bi',
        'analyst bi',
        'bi эксперт',
        'bi-аналитик',
        'powerbi/business intelligence analyst',
        'аналитик bi',
        'специалист по визуализации данных',
        'аналитик внедрения bi-решений',
        'аналитик отчетности bi',
        'аналитик bi',
        'консультант bi',
        'консультант bi/bo',
        'младший bi-аналитик',
        'младший аналитик bi',
        'специалист по bi',
        'старший аналитик bi',
        'специалист визуализации данных',
        'аналитик (визуализация данных + сбор данных из эксель)',
        'аналитик (визуализация данных, создание презентаций и дашбордов)'
    ]
    
    ds_ml = [
    'ml engineer',
    'machine learning developer',
    'разработчик ai',
    'machine learning specialist',
    'software engineer (machine learning)',
    'data scientist',
    'ml-инженер',
    'datascience',
    'ml engineer',
    'machine learning engineer',
    'deep learning',
    'ml/ds',
    'reinforcement',
    'machine learning senior sw engineer',
    'computer vision engineer',
    'специалист по компьютерному зрению',
    'инженер виртуальной реальности',
    'ai and machine learning engineer',
    'ml research engineer',
    'Разработчик имитационных моделей',
    'ml-python разработчик',
    'ds-ml разработчик',
    'python/data modelling developer',
    'quantitative developer',
    'data-scientist',
    'ml аналитик',
    'nlp research',
    'data scientist',
    'ds на проекты по анализу текстов',
    'аналитик ml ',
    'аналитик по моделированию',
    'аналитик, data science',
    'аналитик-эксперт data science',
    'специалист по машинному обучению',
    'специалист прикладной математики и математического моделирования',
    'senior/middle ds',
    'senior ds',
    'middle ds',
    'data science',
    'data science (classic)',
    'data science эксперт',
    'data scientist',
    'аналитик data science',
    'специалист data science',
    'специалист по data science',
    'специалист-математик (data science)',
    'data science intern, bcg gamma',
    'junior специалист data science (python)',
    'research scientist (computer vision)',
    'big data machine learning algorithm researcher',
    'data management algorithms researcher',
    'data sсientist (trade-in)',
    'big data machine learning algorithm researcher',
    'data compression algorithms researcher (math phd)',
    'math libraries (data compression and processing) intern',
    'специалист по интеллектуальному анализу данных',
    'machine learning and computer vision developer',
    'ml разработчик в поиск',
    'разработчик ml',
    'разработчик в области ml',
    'разработчик компьютерного зрения'
    ]
    
    business_analyst = [
        'business analyst',
        'business analytics',
        'it analyst',
        'it business analyst',
        'it business solutions analyst',
        'аналитик по оптимизации бизнес-процессов',
        'аналитик по поддержке бизнес-процессов',
        'бизнес - аналитик',
        'бизнес аналитик',
        'бизнес- аналитик',
        'бизнес-аналитик'
    ]
    
    system_analyst = [
        'system analyst',
        'аналитик (ba/sa)',
        'системный анализ',
        'системный аналитик'
    ]
        
    ds_de = [
    'data engineer', 
    'data инженер', 
    'big data engineer',
    'инженер по обработке данных',
    'Разработчик облачных решений (ориентированных под МО)',
    'azure developer',
    'etl developer',
    'bigdata developer',
    'разработчик dwh',
    'разработчик sql / python',
    'big data - разработчик',
    'big data developer',
    'big data разработчик',
    'developer (big data)',
    'разработчик (big data)',
    'разработчик big data',
    'etl - разработчик',
    'etl / oracle разработчик',
    'etl разработчик',
    'etl- разработчик',
    'etl/bi junior разработчик',
    'etl/bi разработчик',
    'etl/reporting developer',
    'etl-developer',
    'etl-разработчик',
    'sql / etl / bi developer',
    'sql developer (dwh / etl / bi)',
    'инженер данных',
    'разработчик (etl)',
    'разработчик esb / etl',
    'разработчик etl',
    'разработчик etl/bi',
    'разработчик etl/bigdata решений',
    'разработчик etl/esb',
    'разработчик бд и etl',
    'разработчик интеграционных процессов (etl / esb)',
    'bigdata-разработчик',
    'разработчик bigdata',
    'разработчик sql / аналитик (data management, bigdata bi)',
    'Data Quality Engineer',
    'Data-engineer',
    'Data-инженер',
    'DataOps',
    'Data Ops инженер',
    'Data Science Engineer',
    'Data science инженер',
    'big data platform sre',
    'инженер big data',
    'инженер направления big data',
    'инженер по внедрению big data',
    'data - инженер',
    'data operations engineer',
    'data privacy engineer',
    'data quality аналитик',
    'data quality инженер',
    'data quality engineer',
    'data-engineer',
    'dataops specialist',
    'аналитик (data glossary)',
    'аналитик (data governance)',
    'data quality',
    'ведущий специалист по digital решениям (data governance)',
    'data platform engineer',
    'data security engineer',
    'data-инженер',
    'data transformation engineer',
    'инженер big data',
    'инженер dataops',
    'инженер направления big data',
    'инженер по серверам и системам хранения данных',
    'стажер подразделения инфраструктура и система данных',
    'dwh аналитик',
    'аналитик службы управления качеством данных',
    'аналитик (витрины и сервисы озера данных)',
    'аналитик (качество данных)',
    'аналитик витрин данных',
    'аналитик центра обработки, анализа и хранения',
    'аналитик по хранилищам данных'
    ]
    
    db_developer = [    
        'database developer',
        'database intermediate developer',
        'database/dwh developer',
        'developer database',
        'developer цфт-банк pl/sql',
        'ms sql developer',
        'mssql разработчик',
        'mysql developer',
        'mysql/php application developer',
        'pl/sql developer',
        'pl/sql разработчик',
        'sql developer',
        'sql программист',
        'sql разработчик',
        'sql-разработчик',
        't-sql разработчик',
        't-sql-разработчик',
        'главный разработчик pl/sql',
        'программист - разработчик (sql)',
        'программист (oracle',
        'программист (sql',
        'программист ms sql',
        'программист oracle',
        'программист oracle pl/sql',
        'программист pl/sql',
        'программист postgresql',
        'программист sql',
        'программист t-sql',
        'программист-аналитик (oracle pl/sql)',
        'программист-разработчик oracle pl/sql',
        'программист-разработчик sql',
        'разработчик (ms sql)',
        'разработчик (mssql + c#)',
        'разработчик (pl/sql)',
        'разработчик c# и sql',
        'разработчик c#/sql',
        'разработчик greenplum/ms sql',
        'разработчик ms sql',
        'разработчик oracle (ms sql)',
        'разработчик oracle (pl/sql)',
        'разработчик oracle pl/sql',
        'разработчик oracle pl-sql',
        'разработчик oracle/pl-sql',
        'разработчик oracle/sql',
        'разработчик pl/pgsql',
        'разработчик pl/sql',
        'разработчик pl\sql',
        'разработчик pl+ pl/sql',
        'разработчик pl-sql',
        'разработчик postgresql',
        'разработчик sql',
        'разработчик t-sql',
        'разработчик бд',
        'разработчик субд',
        'разработчик хранилища данных',
        'разработчик pl/sql',
        'разработчик-sql',
        'разработчик-эксперт postgresql',
        'oracle ebs developer',
        'oracle db developer',
        'oracle developer',
        'oracle разработчик',
        'dwh developer',
        'sql разработчик',
        'разработчик oracle'

    ]
    
    bi_developer = [
        'bi developer',
        'bi разработчик',
        'bi/sql-разработчик',
        'bi-developer',
        'bi-инженер',
        'bi-разработчик',
        'dwh: разработчик bi',
        'power bi developer',
        'tableau developer',
        'аналитик-программист sql-bi',
        'аналитик-разработчик bi',
        'аналитик-разработчик/bi developer',
        'инженер-программист отдела сопровождения bi',
        'программист - разработчик bi',
        'программист (bi)',
        'программист bi',
        'программист dwh/bi',
        'программист power bi',
        'разработчик bi',
        'разработчик bi&dwh',
        'разработчик oracle bi',
        'разработчик power bi',
        'разработчик tableau',
        'разработчик отчетности (bi)',
        'разработчик отчетности в tableau',
        'разработчик отчетов (oracle bi publisher)',
        'разработчик отчетов power bi',
        'разработчик систем bi',
        'разработчик хранилищ данных / bi',
        'разработчик/аналитик bi (tableau)',
        'разработчик-аналитик sql server, dwh, bi (удаленно)',
        'стажер-разработчик bi',
        'инженер сопровождения dwh/bi',
        'bi engineer',
        'bi developer',
        'bi разработчик',
        'разработчик bi',
        'программист bi',
        'bi/sql-разработчик',
        'dwh developer',
        'sql разработчик',
        'разработчик oracle'
    ]
    
    python = [
        'developer python',
        'django / python разработчик',
        'python / go developer',
        'python developer',
        'python middle developer',
        'python middleware developer',
        'python software engineer',
        'python разработчик',
        'python/django developer',
        'python/django разработчик',
        'python/flask developer',
        'python/go разработчик',
        'python/golang разработчик',
        'python/go-разработчик',
        'python-developer',
        'python-django разработчик',
        'python-разработчик',
        'инженер/разработчик (python)',
        'разработчик (python)',
        'разработчик python',
        'разработчик(developer) python',
        'python & jaicp developer',
        'python (django) программист',
        'python django программист',
        'python программист',
        'software developer (python)',
        'веб-программист python',
        'инженер - программист 2 категории python',
        'инженер-программист (python)',
        'инженер-программист python',
        'инженер-разработчик (программист python)',
        'программист - python',
        'программист (python)',
        'программист python',
        'программист-разработчик чат-ботов на python',
        'разработчик (python',
        'разработчик на python',
        'разработчик-python'
    ]
    
    crm_erp = [
    'crm', 
    'битрикс',
    'bitrix',
    'sap',
    'erp',
    '1c',
    '1с',  # русская эс
    '1c-',
    '1с-',  # русская эс
    '(1c)',
    '(1с)',  # русская эс
    'программист 1 с'
    ]
    
    qa = [
    '(qa)',
    '(тестировщик)',
    '-qa',
    '-тестировщик',
    '/qa',
    '/тестировщик',
    'automation',
    'autoqa',
    'devops&qa',
    'in test',
    'monitoring engineer',
    'playtester',
    'qa',
    'qa-',
    'qa/',
    'qc-engineer',
    'quality assurance',
    'quality engineer',
    'release engineer',
    'site reliability engineer',
    'sqa',
    'ta engineer',
    'test',
    'tester',
    'testing',
    'web-qa', 
    'автоматизатор',
    'автоматизации',
    'автор тестов',
    'автотестам',
    'автотестирование',
    'автотестированию',
    'автотестирования',
    'автотестировщик',
    'автотестов',
    'качества',
    'качеству',
    'пентестер',
    'qa/тестированию'
    'программист-тестировщик',
    'сборщик-тестировщик',
    'тест-',
    'тест-аналитик',
    'тестер',
    'тестирование',
    'тестировании',
    'тестированию',
    'тестирования',
    'тестировщик',
    'тестировщик-',
    'тестировщика'
    ]
    
    dba = [
    'dba',
    'администратор баз данных',
    'администратор базы данных',
    'Администратор бд',
    'Администратор dba',
    'баз данных',
    'администратор данных',
    'администратор субд',
    'database administration',
    'database administrator',
    'database engineer',
    'database administrator'
    ]

    devops = [
    'devops',
    'mlops',
    'devops',
    'devops-',
    '(devops)',
    'девопс',
    'девопс-',
    '(девопс)'
    ]
    
    sys_adm = [
    'системный администратор', 
    'system administrator', 
    'system engineer',
    'администратор linux',
    'системный администратор'
    ]

    network = [
    'администратор вычислительных сетей',
    'администратор сетевого оборудования',
    'администратор сетей',
    'администратор сети',
    'администратор систем хранения данных/сетей',
    'сетевой администратор',
    'сетевой инженер',
    'специалист по сетевому и системному администрированию',
    'network engineer',
    'fixed network service engineer',
    'инженер развития инфраструктуры',
    'network system engineer',
    'netops',
    'infrastructure network engineer']
    
    frontend = [
    'front-end',
    'frontend',
    'front end',
    'фронт-энд',
    'фронт-енд',
    'фронтэнд',
    'фронтенд',
    'фронт энд',
    'фронт енд',
    'фронт-энда',
    'фронт-енда',
    'фронтэнда',
    'фронтенда',
    'фронт энда',
    'фронт енда',
    'вебпрограммист',
    '(front-end)',
    '(frontend)',
    '(front end)',
    '(фронт-энд)',
    '(фронт-енд)',
    '(фронтэнд)',
    '(фронтенд)',
    '(фронт энд)',
    '(фронт енд)',
    '(фронт-энда)',
    '(фронт-енда)',
    '(фронтэнда)',
    '(фронтенда)',
    '(фронт энда)',
    '(фронт енда)',
    'javascript',
    'javascript-',
    '(javascript)',
    'js',
    'js-',
    '(js)',
    'джаваскрипт',
    'джаваскрипт-',
    '(джаваскрипт)',
    'typescript',
    'typescript-',
    '(typescript)',
    'ts',
    'ts-',
    '(ts)',
    'react',
    '(react)',
    '/react',
    'react/',
    'reactjs',
    '(reactjs)',
    '/reactjs',
    'reactjs/',
    'vue.js',
    '(vue.js)',
    '/vue.js',
    'vue.js/',
    'angular',
    '/angular',
    'angular/',
    '(angular)',
    'nuxt.js',
    '(nuxt.js)',
    'd3.js',
    '/d3.js',
    'd3.js/',
    '(d3.js)',
    'frond end developer',
    'fronted-разработчик',
    'frontent-разработчик',
    'frontеnd разработчик',
    'front-еnd разработчик',
    'frontеnd-программист'

    ]

    developer = [
        'программист',
        'разработчик',
        'developer',
        'programmer',
        'developer',
        'software engineer']
    
    backend = [
        'back-end',
        'backend',
        'бэкэнд',
        'бекенд',
        'бэкенд',
        'бекэнд',
        'бэк-энд',
        'бек-енд',
        'бэк-енд',
        'бек-энд',
        'бэкэнда',
        'бекенда',
        'бэкенда',
        'бекэнда',
        'бэк-энда',
        'бек-енда',
        'бэк-енда',
        'бек-энда',
        'backend-',
        'бэкэнд-',
        'бекенд-',
        'бэкенд-',
        'бекэнд-',
        '(back-end)',
        '(backend)',
        '(бэкэнд)',
        '(бекенд)',
        '(бэкенд)',
        '(бекэнд)',
        '(бэк-энд)',
        '(бек-енд)',
        '(бэк-енд)',
        '(бек-энд)',
        'вackend developer',
        'back end developer'
        ]

    fullstack = [
    'fullstack',
    'full-stack',
    'full stack',
    'фуллстек',
    'фулстек',
    'фуллстэк',
    'фулстэк',
    'фулл-стек',
    'фул-стек',
    'фулл-стэк',
    'фул-стэк',
    'фулл стек',
    'фул стек',
    'фулл стэк',
    'фул стэк',
    'fullstack-',
    'фуллстек-',
    'фулстек-',
    'фуллстэк-',
    'фулстэк-',
    '(fullstack)',
    '(full-stack)',
    '(full stack)',
    '(фуллстек)',
    '(фулстек)',
    '(фуллстэк)',
    '(фулстэк)',
    '(фулл-стек)',
    '(фул-стек)',
    '(фулл-стэк)',
    '(фул-стэк)',
    '(фулл стек)',
    '(фул стек)',
    '(фулл стэк)',
    '(фул стэк)',
    'back/front programmer'
    ]
    
    desktop = [
    'desktop',
    'desktop-',
    '(desktop)',
    'десктоп',
    'десктоп-',
    '(десктоп)',
    'десктопных',
    'десктопные'
    ]
    
    abap = [
        'abap разработчик',
        'abap- разработчик',
        'abap-разработчик',
        'abap developer',
        'разработчик abap',
        'abap программист',
        'программист abap'
    ]

    ios = [
        '(ios) разработчик',
        'ios разработчик',
        'associate software engineer (ios)',
        'developer (c++, ios)',
        'developer ios',
        'ios - developer',
        'ios - разработчик',
        'ios developer',
        'ios junior',
        'ios junior developer',
        'ios middle developer',
        'ios программист',
        'ios разработчик',
        'ios/flutter developer',
        'ios-developer',
        'ios-программист',
        'ios-разработчик',
        'software engineer (ios)',
        'программист ios',
        'программист на ios',
        'разработчик ios',
        'разработчик swift (ios/macos)',
        'разработчик под ios',
        'ios ‎разработчик',
        'ios разработчик',
        'стажер-разработчик (приложения под ios)'
    ]
    
    android = [
        'android - разработчик',
        'android (aosp) system software engineer',
        'android (kotlin) developer',
        'android (kotlin) разработчик',
        'android (senior) developer',
        'android developer',
        'android junior developer',
        'android middle developer',
        'android middle/senior developer',
        'android reverse engineer',
        'android software engineer',
        'android программист',
        'android- программист',
        'android разработчик',
        'android -разработчик',
        'android- разработчик',
        'android-developer',
        'android-developer ',
        'android-java разработчик',
        'android-программист',
        'android-разработчик',        
        'c/c++/android ndk developer',
        'flutter (dart ) android/io mobile-developer',
        'flutter (dart, kotlin )/mobile developer',
        'flutter/android разработчик',
        'high middle/senior android developer',
        'java developer for mobile services',
        'java/kotlin разработчик',
        'mobile cpu software r&d engineer',
        'mobile dev (flutter, dart)',
        'unity developer (android)',
        'андроид разработчик',
        'программист android',
        'программист на android',
        'программист приложений для android',
        'программист -разработчик (android)',
        'программист/ разработчик android, java',
        'программист-разработчик (android)',
        'программист-разработчик (mobile android разработчик)',
        'разработчик android',
        'разработчик android приложений (kotlin)',
        'разработчик под android',
        'разработчик под kotlin',
        'разработчик приложений на базе android',
        'andriod разработчик',
        'mobile engineer (java, kotlin)',
        'инженер- программист (java/ android)',
        'разработчик ‎android',
        'разработчик (приложения под android)',
    ]
    
    c_sharp = [
        '(c#) developer',
        '(c#) middle / senior разработчик',
        '(c#) разработчик',
        '.net core c# developer',
        '.net разработчик (программист c#)',
        '/ senior c# developer (unity, research & development)',
        '/ senior c# разработчик',
        'asp.net/c# developer',
        'c# (.net) developer',
        'c# (asp.net) developer',
        'c# .net core developer',
        'c# .net core разработчик',
        'c# .net developer',
        'c# .net программист',
        'c# .net разработчик',
        'c# / wpf разработчик',
        'c# /.net developer',
        'c# /.net developer (удаленная работа)',
        'c# asp.net core ведущий разработчик',
        'c# asp.net core разработчик (ит-разработка и консалтинг для крупного бизнеса)',
        'c# asp.net разработчик',
        'c# developer',
        'c# developer (защита виртуальных сред).',
        'c# developer (переход на kotlin)',
        'c# fullstаck developer',
        'c# junior/middle разработчик',
        'c# middle+/senior developer',
        'c# programmer',
        'c# web developer',
        'c# разработчик',
        'c#- разработчик',
        'c# разработчик .net',
        'c# разработчик в fiji',
        'c#/. net разработчик',
        'c#/.net core developer',
        'c#/.net core middle разработчик',
        'c#/.net core разработчик',
        'c#/.net junior developer',
        'c#/.net software developer',
        'c#/.net разработчик',
        'c#\ .net junior developer',
        'c#\ .net middle developer',
        'c#-разработчик',
        'c++/ c# разработчик',
        'developer (c# .net )',
        'developer (c# программист)',
        'developer (c#)',
        'developer (c#, blazor)',
        'developer .net (c#)',
        'developer .net c# wpf',
        'developer c#',
        'developer) c#/.net',
        'junior (asp.net mvc, sql, c#) developer',
        'research developer (c#)',
        'software developer c#',
        'unity c# developer (удаленная работа, санкт-петербург)',
        'unity c# developer (удаленно)',
        'инженер-программист (c#)',
        'инженер-программист (разработчик c#/wpf)',
        'инженер-программист 1 категории (c#, wpf)',
        'инженер-программист c#',
        'программист - разработчик c#',
        'программист .net ( c# )',
        'программист .net c#',
        'программист .net, c#',
        'программист .net/c#',
        'программист asp.net / c#',
        'программист c#',
        'программист игровой логики c#',
        'программист разработчик .net core / c#',
        'программист-стажер c#',
        'программист-стажёр c# .net',
        'разработчик (c# .net)',
        'разработчик (c#)',
        'разработчик (developer) c#',
        'разработчик (junior) golang. c#',
        'разработчик (mssql + c#)',
        'разработчик .net (c#)',
        'разработчик .net (c#, winforms)',
        'разработчик .net / c#',
        'разработчик .net c#',
        'разработчик .net core/c#',
        'разработчик .net/c#',
        'разработчик asp.net/c#',
        'разработчик c#',
        'разработчик c# .net',
        'разработчик middle .net (c#) developer',
        'разработчик oracle/c# / developer oracle/c#',
        'разработчик по роботизации процессов rpa, c#',
        'разработчик по/c# developer',
        'разработчик прикладной серверной инфраструктуры (c#)',
        'разработчик с++/c# (нейронные сети)',
        'разработчик серверных решений (c#/ ms sql/ postgre sql)',
        'разработчик. net c# ( junior, junior+)',
        'разработчик/ developer c#',
        'разработчик/программист c#',
        'с# .net core разработчик | c# .net core developer',
        'серверный программист c#',
        'стажер c# разработчик',
        'стажер-программист asp.net (c#)',
        'стажер-программист c#',
        'стажер-разработчик c#',
        'developer с#',
        'junior с# (asp.net/xamarin) developer',
        'инженер-программист (с#',
        'инженер-программист 1 категории (разработка по с# )',
        'инженер-программист с#',
        'программист (разработчик) с#',
        'программист .net (с#)',
        'программист с#',
        'программист_ с#',
        'программист-разработчик с#',
        'разработчик .net (с#)',
        'разработчик asp.net с#',
        'разработчик на с#',
        'разработчик с#',
        'разработчик- с#',
        'разработчик/ведущий с# .net core',
        'с# developer',
        'с# программист',
        'с# разработчик',
        'стажер-разработчик с#',
        'старший разработчик с# asp.net'
    ]
    
    php = [
        'developer (php)',
        'middle php developer',
        'php – программист',
        'php - разработчик',
        'php (junior) разработчик',
        'php (laravel) разработчик',
        'php (symfony/laravel) разработчик',
        'php / laravel developer',
        'php application developer',
        'php codeigniter middle developer',
        'php developer',
        'php developer (remote)',
        'php developer (saas)',
        'php developer (старший php-разработчик)',
        'php developer / php программист',
        'php magento 2 developer',
        'php middle developer',
        'php middle разработчик',
        'php web developer',
        'php web-разработчик',
        'php yii2 developer',
        'php yii2 программист',
        'php программист',
        'php разработчик',
        'php- разработчик',
        'php/go программист',
        'php/golang developer',
        'php/laravel developer',
        'php/web-программист',
        'php/yii разработчик',
        'php-developer',
        'php-программист',
        'php–программист',
        'php-разработчик',
        'php-разработчик (infra operations.team)',
        'php-разработчик (удаленно)',
        'php-разработчик в международный проект (удаленно)',
        'web developer (php)',
        'веб программист (фронт) php',
        'веб-разработчик php',
        'программист (php',
        'программист / php',
        'программист php',
        'программист/php',
        'разработчик (php',
        'разработчик php',
        'разработчик на php',
        'рнр - разработчик'

    ]
    
    golang = [
        'developer go',
        'go (golang) developer',
        'go (golang) разработчик',
        'go developer',
        'go golang разработчик',
        'go lang, mqtt программист',
        'go pазработчик',
        'golang - разработчик',
        'golang developer',
        'golang middle developer',
        'golang middle разработчик',
        'golang senior-developer',
        'golang разработчик',
        'golang-developer',
        'golang-разработчик',
        'go-разработчик',
        'software engineer (golang)',
        'инженер программист (golang)',
        'программист golang',
        'разработчик go',
        'разработчик golang',
        'разработчик микросервисов (golang',
        'разработчик на golang'
    ]
    
    embedded = [
    'embedded',
    'эмбеддед',
    'embedded-',
    'эмбеддед-',
    'встраиваемых систем',
    'system programmer',
    'системный программист',
    'system developer',
    'системный разработчик',
    '(embedded)',
    '(эмбеддед)',
    'opencv',
    '/opencv',
    'opencv/',
    '(opencv)',
    'плис',
    'fpga',
    '(embeded)'
    ]

    gamedev = [
        'unity developer',
        'developer unity',
        'game developer',
        'unity 3d developer ',
        'unity middle developer',
        'unity ui programmer',
        'unity ui-разработчик',
        'unity разработчик',
        'unity3d developer',
        'unity3d network developer',
        'unity3d vr game developer',
        'unity-разработчик',
        'игровой разработчик',
        'разработчик 3d игр unity',
        'разработчик unity',
        'разработчик игр на unity',
        'юнити-разработчик junior'
    ]

    java = [
        'atlassian-разработчик (java)',
        'developer (java)',
        'java - разработчик',
        'java / kotlin developer',
        'java developer',
        'java esb разработчик',
        'java junior developer',
        'java junior software developer',
        'java junior разработчик',
        'java middle developer',
        'java software developer',
        'java software engineer',
        'java разработчик',
        'java developer',
        'java/groovy разработчик',
        'java/kotlin developer',
        'java/kotlin разработчик',
        'java-developer',
        'javafx разработчик',
        'java-разработчик',
        'java algo hft Developer',
        'java engineer',
        'разработчик java / python',
        'разработчик junior python',
        'разработчик middle java/python',
        'senior java engineer',
        'production reliability engineer (java/web)',
        'java / groovy developer',
        'java / kotlin разработчик',
        'java junior программист',
        'java инженер-программист',
        'java программист',
        'java- разработчик',
        'java серверный разработчик',
        'java/kotlin-developer',
        'java/kotlin-разработчик',
        'java/web developer',
        'java/web разработчик',
        'java-web developer',
        'java-программист',
        'software engineer (java)',
        'software engineer / java',
        'software engineer java',
        'инженер-программист java',
        'инженер-программист, разработчик java',
        'математик-программист (java)',
        'программист (разработчик) на java',
        'программист java',
        'программист java / kotlin',
        'программист java/kotlin',
        'программист-разработчик java',
        'разработчик (java',
        'разработчик java',
        'разработчик интеграционных решений на платформе java',
        'разработчик информационных систем (java)',
        'разработчик направления java',
        'разработчик под монетизацию (java)'

    ]
    
    c_pp = [
        ' c++ software developer',
        ' c++/qt developer',
        'c/c++ developer',
        'c/c++ linux developer',
        'c/c++ разработчик',
        'c++ (qt) разработчик',
        'c++ (unreal engine) developer',
        'c++ developer',
        'c++ developer (mystery manor)',
        'c++ engine programmer',
        'c++ gameplay programmer',
        'c++ programmer',
        'c++ software developer',
        'c++ tools programmer',
        'c++ программист',
        'c++ разработчик',
        'c++/qt разработчик (qml)',
        'developer (c/c++)',
        'developer (c/c++), роботизированные системы',
        'developer (c++, uwp)',
        'developer (python, c/c++), роботизированные системы',
        'developer / программист c++ (qt)',
        'developer c/c++',
        'developer c/c++, qt',
        'developer c++',
        'developer unreal engine c++',
        'junior c++ developer',
        'programmer c++',
        'qt/c++ developer',
        'software developer (c/c++)',
        'software developer (c++; vantage)',
        'software developer (ocr, c++, ml)',
        'software developer c++',
        'software developer c++/qt',
        'software engineer (c/c++)',
        'software engineer (c++',
        'sw tool developer (c++)',
        'ue4 developer c++/blueprint',
        'ui-разработчик c++ (allods team)',
        'ui-разработчик c++ (warface)',
        'инженер - программист 2 категории (c++, qt и boost)',
        'инженер - программист c++',
        'инженер-программист (c++',
        'инженер-программист / developer qt (c++)',
        'инженер-программист c/c++',
        'инженер-программист c++',
        'программист 3d-симуляторов (c++',
        'программист c / c++',
        'программист c#/c++',
        'программист c/c++',
        'программист c++',
        'программист ui (c++/ue4)',
        'программист встроенного по на языке c\c++',
        'программист встроенных систем (язык c / c++',
        'программист с# c++',
        'программист с/c++',
        'программист с++',
        'программист/разработчик c++',
        'программист-математик с/c++',
        'программист-математик, c#/c++',
        'программист-разработчик (stm32, c/c++)',
        'программист-разработчик c++/qt',
        'разработчик (c++, ml)',
        'разработчик (с/c++)',
        'разработчик c# / c++',
        'разработчик c/c++',
        'разработчик c++',
        'разработчик sdk (c++)',
        'разработчик высокопроизводительных c++',
        'разработчик на c++',
        'разработчик с#/c++',
        'разработчик с/c++',
        'с/c++ developer',
        'сетевой программист c++',
        'developer с/с++', # русское с тут и ниже
        'developer с++',
        'software developer (с++)',
        'инженер - программист 2 категории с++ ',
        'инженер - программист с, с++',
        'инженер-программист c, с++',
        'инженер-программист c/с++',
        'инженер-программист qt, с++',
        'инженер-программист в службу технической поддержки с++ ',
        'инженер-программист встроенного по (c/с++)',
        'инженер-программист с/с++',
        'инженер-разработчик с++',
        'программист (разработчик) на с++',
        'программист / программист-разработчик (с++',
        'программист c/с++',
        'программист микроконтроллеров на языке с\с++',
        'программист микроконтроллеров с/с++',
        'программист на си и с++',
        'программист с#/с++',
        'программист с/с++',
        'программист-алгоритмист с++',
        'программист-разработчик (с++)',
        'программист-разработчик с++',
        'программист-стажер с/с++',
        'разработчик c/с++',
        'разработчик мобильных приложений (с++)',
        'разработчик на с++',
        'разработчик с / с++ ',
        'разработчик с/с++',
        'разработчик с++',
        'разработчик с++ ',
        'разработчик с++\qt',
        'с/с++ developer',
        'с++ developer',
        'с++ qt/qml разработчик',
        'с++ software developer',
        'с++ software engineer',
        'с++ sound engine programmer',
        'с++ unreal engine developer',
        'с++ программист',
        'с++ разработчик',
        'технический программист с#/с++'

    ]
    
    it_support = ['cпециалист для поддержки игрового софта',
                 'cпециалист по поддержке',
                 'cпециалист поддержки',
                 'helpdesk',
                 'it - специалист (м. домодедовская)',
                 'it assistant',
                 'it engineer поддержки финансовых систем',
                 'it help desk engineer',
                 'it junior specialist',
                  'it specialist',
                 'it support',
                 'it инженер',
                 'it поддержки',
                 'it специалист',
                 'it- специалист',
                 'it-cпециалист (it-specialist)',
                 'it-инженер, телекоммуникации (стажер)',
                 'it-консультант',
                 'it-специалист',
                 'it/ot специалист / it/ot specialist',
                 'it\xa0специали\u200b\u200b\u200b\u200b\u200b\u200b\u200bст / инженер',
                 'junior / middle инженер (отдел поддержки)',
                 'junior – инженер 1 линии поддержки',
                 'service desk',
                 'support engineer',
                 'support specialist / специалист клиентской поддержки',
                 'support specialist / специалист по поддержке пользователей (удаленно)',
                 'system monitoring engineer (customer support)',
                 'technical support specialist',
                 'user support',
                 'vip инженер по поддержке пользователей',
                 'администратор (отдел сопровождения информационных технологий)',
                 'администратор 2-й линии поддержки',
                 'администратор программно-аппаратных комплексов уц',
                 'администратор технического сопровождения курсов',
                 'ассистент отдела технического саппорта',
                 'ведущий специалист / специалист поддержки (calypso)',
                 'ведущий специалист по поддержке пользователей',
                 'ведущий специалист по тестированию требований',
                 'ведущий специалист поддержки по',
                 'главный инженер ит поддержки (со знание английского языка)',
                 'главный специалист поддержки уровня 2+',
                 'дежурный it-специалист',
                 'дежурный инженер поддержки приложений',
                 'дежурный инженер-инженер 1 линии поддержки',
                 'диспетчер в it компанию',
                 'диспетчер сервисной поддержки',
                 'инженер (1-я линия поддержки)',
                 'инженер (2 линия поддержки)',
                 'инженер / специалист первой линии поддержки ит-сервисов\u200b\u200b\u200b\u200b\u200b\u200b\u200b',
                 'инженер 2 линии поддержки',
                 'инженер 2-й линии поддержки (support engineer)',
                 'инженер 3 линии поддержки',
                 'инженер help desk',
                 'инженер it - сервиса',
                 'инженер support',
                 'инженер базисной поддержки',
                 'инженер в магазин компьютерной техники',
                 'инженер в отдел мониторинга тв и телематики ( удаленно)',
                 'инженер в отдел сервисной поддержки',
                 'инженер в отдел технических расчетов и поддержки бд, операционный центр ran',
                 'инженер в техническую поддержку',
                 'инженер группы support',
                 'инженер данных (удаленно)',
                 'инженер мониторинга эфира отт',
                 'инженер направления поддержки сервиса',
                 'инженер он-сайт поддержки',
                 'инженер оператор 2-й линии поддержки',
                 'инженер оператор контакт-центра (начинающий специалист)',
                 'инженер отдела клиентской поддержки l2',
                 'инженер отдела сопровождения программных продуктов(удаленно)',
                 'инженер первой линии поддержки',
                 'инженер по it оборудованию и программному обеспечению',
                 'инженер по внедрению и сопровождению (мобильные приложения)',
                 'инженер по поддержке',
                 'инженер по сопровождению',
                 'инженер поддержки',
                 'инженер серверной поддержки',
                 'инженер сервисного центра/it-специалист',
                 'инженер сервисной поддержки',
                 'инженер службы поддержки',
                 'инженер сопровождения (платежи и переводы)',
                 'инженер технического сопровождения (2 линия)',
                 'инженер технической подддержки',
                 'инженер третьей линии поддержки рабочих мест vdi',
                 'инженер удаленной поддержки',
                 'инженер-тестировщик пк, серверов (it оборудования)',
                 'инженер/направление поддержки внутренних клиентов',
                 'ит-специалист',
                 'компьютерный мастер / начинающий специалист по ремонту компьютеров ноутбуков',
                 'консультант в сфере it',
                 'консультант отдела сопровождения пользователей (1 линия)',
                 'консультант по информационным системам',
                 'консультант по поддержке информационных систем',
                 'консультант по продажам (информационные it платформы)',
                 'консультант по сопровождению программного обеспечения',
                 'консультант поддержки web-порталов',
                  'координатор технической группы',
                 'мастер по ремонту iphone / телефонов / ноутбуков (apple / android)',
                 'менеджер в it компанию',
                 'менеджер информационных систем',
                 'менеджер отдела поддержки',
                 'специалист по поддержке',
                 'менеджер по информационным системам',
                 'менеджер по предпродажной поддержке/технический специалист',
                 'менеджер по программному обеспечению fidelio, мicros, tng',
                 'менеджер по продажам ит (компьютеры и оргтехника)',
                 'менеджер по сопровождению клиентов (it',
                 'менеджер по учету it оборудования',
                 'младший инженер itsm',
                 'младший инженер дата центра',
                 'младший инженер по сопровождению',
                 'младший инженер сопровождения (web-разработка)',
                 'младший консультант отдела внедрения и сопровождения информационной системы',
                 'младший консультант поддержки axapta',
                 'младший сетевой инженер (стажер)',
                 'младший специалист по информационной безопасности',
                 'младший специалист поддержки',
                 'младший технический специалист - sam',
                 'мобильный сервисный инженер it',
                 'мобильный сервисный инженер it (обслуживание тс пятерочка\\перекресток)',
                 'начинающий cпециалист контактного центра (it-услуги)',
                 'начинающий it - специалист / технический консультант',
                 'начинающий it-специалист',
                 'начинающий it-специалист/it-инженер',
                 'начинающий инженер отдела внедрения (it-компания лемма)',
                 'начинающий системный инженер',
                 'начинающий специалист / компьютерщик / специалист по it',
                 'начинающий специалист / технический специалист',
                 'начинающий специалист it',
                 'начинающий специалист it по обслуживанию банкоматов',
                 'начинающий специалист it/it инженер',
                 'начинающий специалист it/it-инженер',
                 'начинающий специалист it|it-инженер',
                 'начинающий специалист в it-компанию',
                 'начинающий специалист по информационной безопасности',
                 'начинающий специалист по ремонту компьютеров и ноутбуков/ компьютерный мастер',
                 'начинающий специалист связи (инсталлятор)',
                 'ночной специалист (системное администрирование)/стажер',
                 'ночной специалист по мониторингу ит систем',
                 'ночной специалист технической поддержи (удаленно)',
                 'ночной специалист/стажер (системное администрирование)',
                 'оператор (техническая служба интернет-провайдера)',
                 'оператор компьютерных систем',
                 'оператор по мониторингу ит-систем',
                 'поддержки пользователей',
                 'помощник в отдел erp систем/специалист первой линии поддержки',
                 'помощник системного администратора (1 линия поддержки)',
                 'помощник системного администратора, ассистент it отдела, поддержка help desk',
                 'помощник специалиста в отдел it',
                 'сборщик пк/компьютеров (серверное оборудование)',
                 'сервис-менеджер поддержки',
                 'сервисный инженер (служба поддержки)',
                 'сервисный специалист ( ит отдел)',
                 'сервисный специалист it',
                 'сотрудник отдела информационных технологий',
                 'сотрудник по продаже и ремонту компьютеров',
                 'сотрудник сервисной поддержки клиентов',
                 'специалист (информационные технологии)',
                 'специалист (поддержка 1 линия)',
                 'специалист (резервного копирования)',
                 'специалист (центр технического обслуживания по направлению ит)',
                 'специалист 1-й линии поддержки (без опыта)',
                 'специалист 1-ой линии поддержки он-лайн систем',
                 'специалист 2-й линии тех.поддержки (по)',
                 'специалист 3-й линии поддержки',
                 'специалист help desk',
                 'специалист it',
                 'специалист servicedesk',
                 'специалист sql (2-3 линия поддержки)',
                 'специалист windows',
                 'специалист в it отдел',
                 'специалист в группу поддержки фронт-офисных систем',
                 'специалист в отдел информационных систем',
                 'специалист в региональную техническую поддержку',
                 'специалист в службу поддержки поисковых сервисов',
                 'специалист внедрения',
                 'специалист внутренней поддержки',
                 'специалист второй линии поддержки',
                 'специалист выездной поддержки магазинов',
                 'специалист группы мониторинга сервисов',
                 'специалист группы операционной поддержки',
                 'специалист группы поддержки',
                 'специалист информационно-технического отдела',
                 'специалист информационной поддержки',
                 'специалист информационных технологий',
                 'специалист ит',
                 'специалист компьютерных сетей и техники / специалист it отдела',
                 'специалист личной поддержки',
                 'специалист мониторинга и сопровождения систем',
                 'специалист на поддержку системы альфа',
                 'специалист отдела it',
                 'специалист отдела валидации компьютеризированных систем',
                 'специалист отдела внедрения',
                 'специалист отдела внедрения и сопровождения',
                 'специалист отдела внедрения по',
                 'специалист отдела информационно-технического сопровождения',
                 'специалист отдела ит-поддержки',
                 'специалист отдела качества ит',
                 'специалист отдела клиентского сопровождения (it)',
                 'специалист отдела компьютерного обеспечения/монтажник лвс',
                 'специалист отдела обслуживания средств связи',
                 'специалист отдела поддержки',
                 'специалист отдела сопровождения пользователей',
                 'специалист отдела технического сопровождения',
                 'специалист первой линии поддержки информационных систем',
                 'специалист письменной поддержки в магазинах приложений (кинопоиск, музыка, яндекс.афиша, плюс)',
                 'специалист по it',
                 'специалист по внедрению и сопровождению',
                 'специалист по внедрению и сопровождению',
                 'специалист по внедрению по',
                 'специалист по внедрению программного обеспечения',
                 'специалист по внедрению/ сопровождению по',
                 'специалист по выявлению компьютерных атак и реагированию на компьютерные инциденты',
                 'специалист по информационной поддержке',
                 'специалист по информационным системам',
                 'специалист по информационным технологиям',
                 'специалист по информационным технологиям - консультант',
                 'специалист по ит',
                 'специалист по ккт (контрольно-кассовая техника)',
                 'специалист по компьютерам и связи',
                 'специалист по компьютерному оборудованию',
                 'специалист по лис',
                 'специалист по навигации и обслуживанию оборудования (специалист по информационным технологиям)',
                 'специалист по настройке и внедрению по',
                 'специалист по настройке ит оборудования',
                 'специалист по настройке программного обеспечения',
                 'специалист по обслуживанию',
                 'специалист по офисным стандартным, мобильным и прочим информационным системам',
                 'специалист по подбору компьютерного оборудования и оргтехники',
                 'специалист по поддержке',
                 'специалист по программному обеспечению',
                 'специалист по продвижению инженерного программного обеспечения',
                 'специалист по работе с',
                 'специалист по ремонту компьютеров и компьютерной техники',
                 'специалист по ремонту пк',
                 'специалист по ремонту пк и сети',
                 'специалист по сервисной поддержке',
                 'специалист по системной поддержке',
                 'специалист по сопровождению',
                 'специалист по телекоммуникациям',
                 'специалист по технической документации в ит',
                 'специалист по техническому сопровождению',
                 'специалист по технологиям',
                 'специалист поддержки',
                 'специалист системной поддержки',
                 'специалист службы информационной поддержки',
                 'специалист службы информационных технологий',
                 'специалист службы локальной ит-поддержки',
                 'специалист службы мониторинга (ночные смены)',
                 'специалист службы мониторинга и 2 линии поддержки (удаленно)',
                 'специалист службы поддержки',
                 'специалист службы сервисной поддержки',
                 'специалист службы технической и методологической поддержки',
                 'специалист сопровождения',
                 'специалист технический поддержки (1 линия)',
                 'специалист технического',
                 'специалист технической поддержи',
                 'специалист управления технического и программного обеспечения',
                 'специалист учета it-оборудования',
                 'специалист центра мониторинга и управления нагрузкой',
                 'специалист-инженер по сопровождению программно-технического комплекса (тренажёры)',
                 'стажер it service academy',
                 'стажер мониторинга сервисов и систем отдела технического контроля',
                 'стажер службы поддержки в области it',
                 'стажер специалиста информационной безопасности',
                 'стажёр-специалист по компьютерным сетям',
                 'старший специалист в сервис яндекс.еда',
                 'старший специалист по поддержке пользователей',
                 'старший специалист по управлению контентом в информационных сервисах',
                 'старший специалист поддержки',
                 'старший специалист сервисной поддержки',
                 'техник в отдел информационных технологий',
                 'техник информационных технологий',
                 'техник отдела информационно-технического сопровождения',
                 'техник отдела информационных технологий',
                 'техническая поддержка',
                 'технический интегратор/ integrator-scripter',
                 'технический копирайтер в it',
                 'технический специалист (sam)',
                 'технический специалист (оператор связи)',
                 'технический специалист (стажер)',
                 'технический специалист на спортивном мероприятии (эникей, it)',
                 'технический специалист онлайн проекта',
                 'технический специалист по настройке vr-тренажёров',
                 'технический специалист по сопровождению',
                 'технический специалист по сопровождению по model studio cs cadlib модель и архив',
                 'технический специалист поддержки выездного персонала',
                 'технический специалист поддержки клиентов',
                 'технический специалист яндекс.директ',
                 'технический специалист, череповец',
                 'технический специалист/начинающий специалист',
                 'технической поддержке',
                 'технической поддержки',
                 'техподдержка',
                 'техподдержки']
    
    exclude = [
         'teamlead',
         'team lead',
         'преподаватель',
         'руководитель',
         'architect',
         'архитектор'
         'evangelist',
         'технический писатель',
         'методист',
         'автор работ по',
         'ментор',
         'педагог',
        'менеджер в отдел веб-разработки',
        'наставник курса',
        'руководитель',
        'директор',
        'сhief technical officer',
        'director',
        'руководитель',
        'менеджер по продажам',
        'marketing manager',
        'маркетинг',
        'маркетолог',
        'маркетинга',
        'аккаунт менеджер',
        'контент-менеджер',
        'аккаунт-менеджер',
        'контент менеджер',
        'crm-менеджер (обучение)',
        'product manager',
        'product owner',
        'менеджер crm',
        'специалист crm и рекламных площадок',
        'менеджер по продаже',
        'менеджер crm кампаний',
        'со знанием crm',
        'crm менеджер',
        'crm - менеджер',
        'crm-менеджер',
        'crm lead',
        'senior analyst (e-commerce/crm)',
        'crm specialist',
        'head of',
        'менеджер по работе с клиентами',
        'account manager',
        'экономист',
        'архитектор',
        'заместитель руководителя',
        'менеджер по развитию клиентов',
        'менеджер проектов',
        'начальник отдела',
        'взыскания',
        'задолженности',
        'account Lead',
        'acoustic measurements specialist',
        'after sales spare parts specialist',
        'team lead analyst',
        'администратор проекта (big data)',
        'менеджер по внедрению продуктов больших данных',
        'эксперт/директор по анализу данных',
        'младший технический менеджер направления',
        'руководитель проектов (в направлении bi аналитика, data аналитика)',
        'head of data science',
        'ml/data science team lead',
        'team lead analyst (big data)',
        'team lead data scientist',
        'team-lead бизнес-аналитик (sql)',
        'архитектор bi',
        'наставник',
        'код-ревьюер на курс '
    ]
    
    if any(x in row['name'].lower() for x in exclude):
        val = 'Исключить'

    elif any(x in row['name'].lower() for x in bi_analyst):
        val = 'BI Analyst'                 
    
    elif any(x in row['name'].lower() for x in game_prod_web_da):
        val = 'Game/Product/Web Analyst'        
    
    elif any(x in row['name'].lower() for x in data_analyst):
        val = 'Data Analyst'    

    elif any(x in row['name'].lower() for x in business_analyst):
        val = 'Business Analyst'        
    
    elif any(x in row['name'].lower() for x in system_analyst):
        val = 'System Analyst'    

    elif any(x in row['name'].lower() for x in ds_de):
        val = 'Data Engineer'
        
    elif any(x in row['name'].lower() for x in ds_ml):
        val = 'Data Science (ML)'

    elif any(x in row['name'].lower() for x in qa):
        val = 'Тестировщик'           
        
    elif any(x in row['name'].lower() for x in crm_erp):
        val = 'CRM, ERP'    
    
    elif any(x in row['name'].lower() for x in bi_developer):
        val = 'Программист, разработчик (BI)'
        
    elif any(x in row['name'].lower() for x in db_developer):
        val = 'Программист, разработчик (DB)'   
        
    elif any(x in row['name'].lower() for x in dba):
        val = 'Администратор баз данных'        
        
    elif any(x in row['name'].lower() for x in devops):
        val = 'DevOps'     

    elif any(x in row['name'].lower() for x in sys_adm):
        val = 'Системный администратор'

    elif any(x in row['name'].lower() for x in network):
        val = 'Сетевой инженер'           
        
    elif any(x in row['name'].lower() for x in android):
        val = 'Программист, разработчик (Android)' 
        
    elif any(x in row['name'].lower() for x in ios):
        val = 'Программист, разработчик (iOS)'         
        
    elif any(x in row['name'].lower() for x in c_sharp):
        val = 'Программист, разработчик (C#)'         
        
    elif any(x in row['name'].lower() for x in c_pp):
        val = 'Программист, разработчик (C++)'        

    elif any(x in row['name'].lower() for x in php):
        val = 'Программист, разработчик (PHP)'            

    elif any(x in row['name'].lower() for x in golang):
        val = 'Программист, разработчик (Golang)'         
        
    elif any(x in row['name'].lower() for x in frontend):
        val = 'Программист, разработчик (frontend)'          
                
    elif any(x in row['name'].lower() for x in fullstack):
        val = 'Программист, разработчик (fullstack)'    
                
    elif any(x in row['name'].lower() for x in backend):
        val = 'Программист, разработчик (backend)' 
        
    elif any(x in row['name'].lower() for x in java):
        val = 'Программист, разработчик (Java)' 
        
    elif any(x in row['name'].lower() for x in python):
        val = 'Программист, разработчик (Python)'
                
    elif any(x in row['name'].lower() for x in desktop):
        val = 'Программист, разработчик (desktop)'                  
                
    elif any(x in row['name'].lower() for x in embedded):
        val = 'Программист, разработчик (embedded)'    
                
    elif any(x in row['name'].lower() for x in gamedev):
        val = 'Программист, разработчик (gamedev)' 
        
    elif any(x in row['name'].lower() for x in abap):
        val = 'Программист, разработчик (ABAP)'               
                
    elif any(x in row['name'].lower() for x in developer):
        val = 'Программист, разработчик (other developer)' 
        
    elif any(x in row['name'].lower() for x in it_support):
        val = 'Специалист технической поддержки'  
        
    else:
        val = 'empty'
    
    return val

df['job_family'] = df.apply(my_filter, axis=1)

In [20]:
df.groupby(['job_family'])['id'].count().sort_values(ascending=False)

job_family
empty                                         34894
Исключить                                     11826
Специалист технической поддержки              10231
CRM, ERP                                       4677
Тестировщик                                    4332
Программист, разработчик (other developer)     3379
Программист, разработчик (frontend)            2640
Системный администратор                        2393
System Analyst                                 1475
Программист, разработчик (Java)                1198
Программист, разработчик (PHP)                 1129
DevOps                                          866
Business Analyst                                839
Программист, разработчик (backend)              715
Программист, разработчик (C#)                   690
Программист, разработчик (Android)              668
Data Analyst                                    587
Программист, разработчик (iOS)                  548
Программист, разработчик (Python)               545
П

In [21]:
professions = list(df['job_family'].unique())
professions.sort()
for el in professions:
    print(el, end=', ')

BI Analyst, Business Analyst, CRM, ERP, Data Analyst, Data Engineer, Data Science (ML), DevOps, Game/Product/Web Analyst, System Analyst, empty, Администратор баз данных, Исключить, Программист, разработчик (ABAP), Программист, разработчик (Android), Программист, разработчик (BI), Программист, разработчик (C#), Программист, разработчик (C++), Программист, разработчик (DB), Программист, разработчик (Golang), Программист, разработчик (Java), Программист, разработчик (PHP), Программист, разработчик (Python), Программист, разработчик (backend), Программист, разработчик (desktop), Программист, разработчик (embedded), Программист, разработчик (frontend), Программист, разработчик (fullstack), Программист, разработчик (gamedev), Программист, разработчик (iOS), Программист, разработчик (other developer), Сетевой инженер, Системный администратор, Специалист технической поддержки, Тестировщик, 

In [22]:
professions

['BI Analyst',
 'Business Analyst',
 'CRM, ERP',
 'Data Analyst',
 'Data Engineer',
 'Data Science (ML)',
 'DevOps',
 'Game/Product/Web Analyst',
 'System Analyst',
 'empty',
 'Администратор баз данных',
 'Исключить',
 'Программист, разработчик (ABAP)',
 'Программист, разработчик (Android)',
 'Программист, разработчик (BI)',
 'Программист, разработчик (C#)',
 'Программист, разработчик (C++)',
 'Программист, разработчик (DB)',
 'Программист, разработчик (Golang)',
 'Программист, разработчик (Java)',
 'Программист, разработчик (PHP)',
 'Программист, разработчик (Python)',
 'Программист, разработчик (backend)',
 'Программист, разработчик (desktop)',
 'Программист, разработчик (embedded)',
 'Программист, разработчик (frontend)',
 'Программист, разработчик (fullstack)',
 'Программист, разработчик (gamedev)',
 'Программист, разработчик (iOS)',
 'Программист, разработчик (other developer)',
 'Сетевой инженер',
 'Системный администратор',
 'Специалист технической поддержки',
 'Тестировщик']

In [23]:
# запоминаем количество записей до удаления нерелевантных профессий
initial_shape = df.shape

# список профессий
professions = list(df['job_family'].unique())

# Убираем те профессии,которые нам нерелевантны
professions_research = [el for el in professions if el not in ['empty', 'Исключить']]
assert len(professions) == len(professions_research) + 2

# выведем профессии для анализа
professions_research.sort()
for el in professions_research:
    print(el, sep=',')

# дафафрейм теперь будет включать только релевантные профессии
df = df[df['job_family'].isin(professions_research)]
changed_shape = df.shape

BI Analyst
Business Analyst
CRM, ERP
Data Analyst
Data Engineer
Data Science (ML)
DevOps
Game/Product/Web Analyst
System Analyst
Администратор баз данных
Программист, разработчик (ABAP)
Программист, разработчик (Android)
Программист, разработчик (BI)
Программист, разработчик (C#)
Программист, разработчик (C++)
Программист, разработчик (DB)
Программист, разработчик (Golang)
Программист, разработчик (Java)
Программист, разработчик (PHP)
Программист, разработчик (Python)
Программист, разработчик (backend)
Программист, разработчик (desktop)
Программист, разработчик (embedded)
Программист, разработчик (frontend)
Программист, разработчик (fullstack)
Программист, разработчик (gamedev)
Программист, разработчик (iOS)
Программист, разработчик (other developer)
Сетевой инженер
Системный администратор
Специалист технической поддержки
Тестировщик


In [24]:
assert changed_shape[0] + 34894 + 11826 == initial_shape[0]

In [25]:
vacancies_extract = df.groupby(['job_family'])['id'].count().sort_values(ascending=False)

#### Промежуточные выводы:
- На одну вакансию приходится одна проф.роль и до 6 специализаций.
- Специализации получаются неинформативны для нашего исследования на этапе формирования выборки для исследования.
- В текущую выборку из 87 306 вакансий попали вакансии, которые мы не относим к предмету нашего исследования (например, 'Автослесарь, автомеханик', 'Хостес', 'Повар, пекарь, кондитер').
- Мы составили списки ключевых слов и словосочетаний, чтобы сгруппировать вакансии в принятые в индустрии группы.
- Выборка релевантных вакансий сократилась до 40575 вакансий.
- Мы выделили следующие профессии:
        BI Analyst, 
        Business Analyst, 
        CRM, ERP, 
        Data Analyst, 
        Data Engineer, 
        Data Science (ML), 
        DevOps, 
        Game/Product/Web Analyst, 
        System Analyst, empty, 
        Администратор баз данных, 
        Программист, разработчик (ABAP), 
        Программист, разработчик (Android), 
        Программист, разработчик (BI), 
        Программист, разработчик (C#), 
        Программист, разработчик (C++), 
        Программист, разработчик (DB), 
        Программист, разработчик (Golang), 
        Программист, разработчик (Java), 
        Программист, разработчик (PHP), 
        Программист, разработчик (Python), 
        Программист, разработчик (backend), 
        Программист, разработчик (desktop), 
        Программист, разработчик (embedded), 
        Программист, разработчик (frontend), 
        Программист, разработчик (fullstack), 
        Программист, разработчик (gamedev),
        Программист, разработчик (iOS), 
        Программист, разработчик (other developer), 
        Сетевой инженер, 
        Системный администратор, 
        Специалист технической поддержки, 
        Тестировщик

In [26]:
df.to_csv(os.path.join(path_save, 'df_vFinal.csv'))

### Обрабатываем Key_skills

In [27]:
# df_key_skills = pd.read_csv(os.path.join(path_download, 'vac_key_skills.csv'))
# df_key_skills = df_key_skills.drop(columns=['Unnamed: 0'])

In [28]:
vacancies_research = list(df['id'])
df_key_skills = df_key_skills[df_key_skills['id'].isin(vacancies_research)]

In [29]:
df_skills = df_key_skills.merge(df[['id', 'name', 'area_name', 'address_lat', 'address_lng', 'experience_name', 'schedule_name', 'employment_name', 'employer_id',
       'employer_name', 'published_at', 'alternate_url', 'job_family']], how='right', on='id', left_index=False)

In [30]:
df_skills.groupby(['job_family', 'key_skill'])['id'].count().to_excel(os.path.join(path_save, 'skills.xlsx'))

**Смотрим, у скольких вакансий нет ключевых скиллов**

In [31]:
len(df_skills[df_skills['key_skill'].isna()]['id'].unique()) / df.shape[0] * 100 # no skills in hh initial data

20.810131572463412

In [32]:
missing_skills = df_skills[df_skills['key_skill'].isna()]
missing_skills = missing_skills.groupby(['job_family'])['id'].count().sort_values(ascending=False)

frames = [vacancies_extract.rename('total'), missing_skills.rename('missing')]
result = pd.concat(frames, axis=1)
result['%'] = result['missing'] / result['total'] *100
result.sort_values(by='%', ascending=False)

,total,missing,%
job_family,,,
"Программист, разработчик (desktop)",5,2,40.000000
Специалист технической поддержки,10231,3167,30.954941
Тестировщик,4332,1049,24.215143
Data Engineer,461,97,21.041215
Системный администратор,2393,482,20.142081
Администратор баз данных,287,56,19.512195
"Программист, разработчик (Java)",1198,232,19.365609
"CRM, ERP",4677,903,19.307248
"Программист, разработчик (BI)",193,37,19.170984


In [33]:
df_descr = pd.read_csv(os.path.join(path_download, 'vac_descr.csv'))
df_descr = df_descr.drop(columns=['Unnamed: 0'])

In [34]:
# создаем список уникальный скиллов, имеющихся уже в key_skills
skills = df_skills.loc[df_skills['key_skill'].notna(), 'key_skill']
skills = sorted(skills.unique().tolist())

# создаем словарь для поиска
skills_short = ['R', 'Си', 'BI', 'C#', '1C', '1c', '1С', '1с']

skills_dic = {} # modified_value: key_skills

for skill in skills:
    
    if len(skill) <= 2 and (skill in skills_short):
        adj_skill = (' ' + skill.lower() + ' ')
        skills_dic[adj_skill] = skill
        
    elif len(skill) > 3:
        adj_skill = (skill.replace('"', '')).lower()
        skills_dic[adj_skill] = skill
    else:
        pass
    
# ищем значения в описании вакансий и присваиваем к номеру вакансии ключевой навык
data = df_descr.copy()

data_id = []
data_skill = []

for index, row in data.iterrows():
    descr = str(row[1]).lower()
    for adj_skill, skill in skills_dic.items():
        if adj_skill in descr:            
            data_id.append(row[0])
            data_skill.append(skill)
        else:
            pass

# создаем словарь
data_df = pd.DataFrame(
    {'id': data_id,
     'key_skill': data_skill,
     'source': 'data mining'
    })    

data_df = data_df.drop_duplicates()

In [35]:
# data_df.to_csv(os.path.join(path_save, 'data_df(skills)_lower_2021_11_18.csv'))

In [36]:
df_key_skills['source'] = 'hh.ru API'
frames = [df_key_skills, data_df]
df_skills_full = pd.concat(frames, axis=0)
# 1716450 rows × 3 columns
# 1670795 rows × 3 columns
df_skills_full = df_skills_full.drop_duplicates(subset=['id', 'key_skill'])
df_skills_full.groupby('source')['id'].count()

source
data mining    1479144
hh.ru API       191651
Name: id, dtype: int64

In [37]:
dic_skills = pd.read_excel(os.path.join(path_download, 'skills_analysis update_vFinal.xlsx'))
df_skills_full = df_skills_full.merge(dic_skills, how='left', on='key_skill', right_index=False)
df_skills_full = df_skills_full[['id', 'skill', 'skill_group', 'skill_type', 'source']]
df_skills_full = df_skills_full.drop_duplicates(subset=['id','skill'])

In [38]:
df_skills_full.to_csv(os.path.join(path_save, 'skills_vFinal.csv'))

### Обрабатываем Условия труда

In [72]:
# создаем списки уникальных условий

benefits_insurance = [
  "ДМС (бизнес+) после испытательного срока",
  "ДМС (со стоматологией)",
  "ДМС за счет Компании",
  "ДМС после испытательного срока",
  "ДМС с первого дня",
  "ДМС со стоматологией",
  "ДМС"
  "ДМС",
  "Добровольное медицинское страхование (в том числе стоматология)",
  "Надежная страховка и внимание к здоровью",
  "Отдельное страхование жизни и здоровья",
  "Полис ДМС по окончании испытательного срока (включая стоматологию",
  "добровольного медицинского страхования",
  "медицинская страховка ДМС со стоматологией",
  "медицинскую страховку",
  "страхование для выезжающих за границу",
  "страхование жизни",
  "страхование от несчастных случаев",
  ]

flexible_schedule = [
  "Гибкий график",
  "Гибкое начало рабочего дня", 
  "свободный график",
  ]

agile = [
    "Agile",
    "agile",
    "agile-команд",
    "agile-команду",
    "Работаем по Agile",
    "работа по Agile"   
    "работает по Agile",
    "работы в Agile",
    ]

hourly = [
    "оплата почасовая",
    "сдельная оплата",
    "сдельную оплату ",
    ]
    
shift = [ 
    "работа в вечернее или ночное время",
    "скользящий график 5/2",
    "Сменный режим работы"
    ]
  
motivation_growth = [
    "Возможности для роста и профессиональной самореализации",
    "Возможность профессионально развиваться",
    "Возможность развиваться вместе с нашим бизнесом",
    "Заинтересованы в том, чтобы сотрудники росли и развивались",
    "Интересные и амбициозные задачи",
    "Неограниченные перспективы карьерного и профессионального роста",
    "Отличные возможности для самореализации",
    "Сложные и интересные задачи, которые позволят полностью раскрыть свой потенциал",   
    "возможности для профессионального роста",
    "возможность войти в IT без специального образования и опыта",
    "возможность развития в области Big Data",
    "опытные коллеги", 
    "разноплановые и творческие задачи на выбор", 
    "Корпоративное обучение",
    "корпоративное обучение, конференции, Coursera",
    "получать дополнительное образование для развития hard и soft навыков",
    "постоянное развитие вместе с нашей командой",
    "сертификацию за счет компании",
    "решать множество сложных и нестандартных задач",
    "сложная и интересная тема",
    "сложные и интересные задачи",
    "сложные, амбициозные, интересные проекты",
    "сотрудники росли и развивались",
    "сертификацию за счёт компании",
    "Ежегодные курсы повышения квалификации",
    "Обучение за счёт компании"
    ]  

motivation_modern = [
    "Вести разработку в современном стек", 
    "Возможность работы с новыми технологиями",
    "Возможность работы с новыми технологиями",
    "Работы с очень большими объемами данных",
    "в создании новой логистической IT-системы",
    "работу над высоконагруженным и активно развивающимся сервисом"   
    
    ]
    
motivation_impact = [
    "Минимум бюрократии и максимум возможностей влиять на продукт",
    "Мы используем в работе продуктовый подход",
    "влиять на создание и развитие продукта",
    "Команду, которая верит в то, что делает",
    "Свободу в создании и улучшении процессов",
    "внедрять крутые продуктовые фичи",
    "внести вклад в развитие Chromium",
    "возможность предложить и реализовать свою идею",  
    "возможность с гордостью рассказывать друзьям и близким о своей работе",  
    "развитии новых банковских продуктов",
    "свободу действий в принятии решений",
    "у вас есть шанс встать у истоков проектов",    
    "участвовать в построении самого лучшего клиентского сервиса", 
    "участие в разработке уникальных продуктов"    
    ]
  
non_TK = [
    "сотрудничество по договору гражданско-правового характера",
    "договор гражданско-правового характера",
    "Возможность оформления по ТК РФ или по договору с ГПХ, с самозанятым или ИП",
    "самозанят",
    "ГПХ"
    ]

official_employment = [
  "Официальное оформление",
  "Официальное трудоустройство",
  "Оформление в штат по ТК РФ"
  "Оформление по ТК",
  "Трудоустройство по ТК РФ",
  "Трудоустройство согласно ТК РФ",
  "официальное трудоустройство с первого дня работы",
  "трудоустройство согласно трудовому кодексу",
  ]
  
benefits_mobile = [
  "оплата мобильной связи",
  "Компенсация мобильной связи",
  "Бесплатная сотовая связь",
  "мобильная связь",
  "служебная сотовая связь",
  "Cлужебную сотовую связь"
  ]
  
benefits_remote = [
  "возможна удалённая работа",
  "удаленная работа",
  "работа из дома",
  "Удаленная работа",
  "Удаленную работу",
  "Удаленный режим работы",
  "полностью удалённой работы",
  "возможность выполнять задания из любой точки мира, где есть интернет",
  "свободный выбор места для работы",
  "Возможность работать удаленно",
  "удаленно",
  "Гибкий подход к «удаленке»",
  "гибкий рабочий график и возможность работать удаленно",
  "Гибридный формат работы",
  "в гибридном формате",
  "по гибридному графику"  
  ]


benefits_sport = [
  "регулярные турниры по киберспорту",
  "шахматам",
  "настольному теннису",
  "футболу и волейболу)",
  "волейболу",
  "до 50% на занятия спортом",
  "скидки в фитнес-клубах",
  "фитнес",
  "спорт"
  ]

benefits_english = [
  "изучение английского языка",
  "курсы английского языка для желающих (50% компенсация)",
  "бесплатные курсы английского языка"
  ]

benefits_extra_vacation = [
  "Отпуск 33 календарных дня",
  "дополнительные дни отпуска"
  ]

benefits_insurance = [x.lower() for x in benefits_insurance]
flexible_schedule = [x.lower() for x in flexible_schedule]
agile = [x.lower() for x in agile]
hourly = [x.lower() for x in hourly]
shift = [x.lower() for x in shift]
motivation_growth = [x.lower() for x in motivation_growth]
motivation_modern = [x.lower() for x in motivation_modern]
motivation_impact = [x.lower() for x in motivation_impact]
non_TK = [x.lower() for x in non_TK]
official_employment = [x.lower() for x in official_employment]
benefits_mobile = [x.lower() for x in benefits_mobile]
benefits_remote = [x.lower() for x in benefits_remote]
benefits_sport = [x.lower() for x in benefits_sport]
benefits_english = [x.lower() for x in benefits_english]
benefits_extra_vacation = [x.lower() for x in benefits_extra_vacation]

conditions = {}
for el in benefits_insurance:
    conditions[el] = 'ДМС'
for el in flexible_schedule:
    conditions[el] = 'Гибкий график'
for el in agile:
    conditions[el] = 'Agile-команда'
for el in hourly:
    conditions[el] = 'Почасовая оплата'
for el in shift:
    conditions[el] = 'Посменный график работы'
for el in motivation_growth:
    conditions[el] = 'Профессиональный рост'    
for el in motivation_modern:
    conditions[el] = 'Современный стек'
for el in motivation_impact:
    conditions[el] = 'Влияние на бизнес'
for el in official_employment:
    conditions[el] = 'Оформление по ТК'
for el in benefits_mobile:
    conditions[el] = 'Корпоративный мобильный'
for el in benefits_remote:
    conditions[el] = 'Удаленка/гибрид'   
for el in benefits_sport:
    conditions[el] = 'Корпоративный спорт'
for el in benefits_english:
    conditions[el] = 'Корпоративный английский'
for el in benefits_extra_vacation:
    conditions[el] = 'Доп.дни отпуска'

In [111]:
# ищем значения в описании вакансий и присваиваем к номеру вакансии найденные условия труда 
data = df_descr.copy()

data_id = []
data_conditions = []

for index, row in data.iterrows():
    descr = str(row[1]).lower()
    for cond_descr, cond in conditions.items():
        if cond_descr in descr:            
            data_id.append(row[0])
            data_conditions.append(cond)
        else:
            pass

# создаем словарь
cond_df_by_vacancy = pd.DataFrame(
    {'id': data_id,
     'conditions': data_conditions
    })    

cond_df_by_vacancy = cond_df.drop_duplicates()

In [105]:
temp = cond_df.merge(output_employers)
cond_df_by_employer = pd.DataFrame(temp.groupby(['employer_name', 'conditions'])['id'].count()).drop(columns='id').reset_index()
cond_df_by_employer = cond_df_by_employer.drop_duplicates()

### Выгрузка данных для Business Intelligence

In [114]:
output_vacancies = df[['id', 'name', 'published_at', 'alternate_url', 'job_family', 'experience_name', 'schedule_name', 'employment_name']]
output_employers = df[['id', 'area_name', 'address_lat', 'address_lng', 'employer_id',
       'employer_name']]
output_skills = df_skills_full.copy()
output_skills = output_skills[output_skills['skill_group'] != 'Minor']
output_salary = df[['id', 'name', 'area_name', 'salary_from', 'salary_to', 'salary_ccy', 'salary_gross']]

output_vacancies.to_csv(os.path.join(path_save, 'output_vacancies.csv'))
output_employers.to_csv(os.path.join(path_save, 'output_employers.csv'))
output_skills.to_csv(os.path.join(path_save, 'output_skills.csv'))
cond_df_by_vacancy.to_csv(os.path.join(path_save, 'output_cond_by_vacancy.csv'))
cond_df_by_employer.to_csv(os.path.join(path_save, 'output_cond_by_employer.csv'))
output_salary.to_csv(os.path.join(path_save, 'output_salary.csv'))

In [128]:
output_uniqie_employers = df['employer_id'].unique()
output_uniqie_employers = list(output_uniqie_employers)
output_uniqie_employers.sort()
for el in output_uniqie_employers:
    print(el, end=', ')

0, 6, 13, 15, 19, 25, 41, 44, 51, 54, 59, 65, 76, 80, 89, 115, 125, 139, 154, 162, 172, 198, 201, 228, 229, 247, 250, 257, 272, 274, 281, 301, 320, 321, 325, 330, 336, 339, 343, 356, 370, 381, 389, 393, 399, 402, 404, 406, 407, 429, 434, 456, 468, 470, 476, 490, 522, 533, 536, 537, 563, 566, 568, 569, 586, 589, 590, 599, 606, 616, 624, 625, 626, 662, 673, 675, 678, 679, 693, 702, 722, 723, 733, 738, 740, 760, 779, 784, 798, 802, 816, 822, 841, 849, 854, 865, 882, 889, 898, 947, 950, 953, 955, 956, 960, 979, 993, 1009, 1037, 1057, 1063, 1081, 1095, 1101, 1107, 1115, 1140, 1144, 1149, 1160, 1171, 1180, 1186, 1190, 1196, 1201, 1222, 1231, 1258, 1271, 1276, 1283, 1285, 1292, 1295, 1304, 1329, 1346, 1391, 1417, 1433, 1439, 1449, 1455, 1466, 1481, 1485, 1496, 1549, 1554, 1563, 1587, 1605, 1638, 1697, 1732, 1735, 1740, 1754, 1765, 1785, 1792, 1793, 1808, 1825, 1833, 1844, 1861, 1870, 1873, 1888, 1904, 1925, 1938, 1947, 1978, 1982, 1988, 1994, 2023, 2033, 2047, 2053, 2057, 2060, 2064, 2097, 21

In [109]:
df.head()

,id,name,area_name,salary_from,salary_to,salary_ccy,salary_gross,type_name,address_lat,address_lng,experience_name,schedule_name,employment_name,employer_id,employer_name,published_at,alternate_url,prof_role,job_family
0,14694668,Junior разработчик (стажер),Тула,20000.0,NaN,RUR,0.0,Открытая,54.185099,37.612012,Нет опыта,Полный день,Полная занятость,1196086,ЮниВеб,2021-10-31 19:51:34+03:00,https://hh.ru/vacancy/14694668,"Программист, разработчик","Программист, разработчик (other developer)"
1,14820007,Программист 1С,Тула,70000.0,NaN,RUR,1.0,Открытая,54.182090,37.573411,От 1 года до 3 лет,Полный день,Полная занятость,200522,"СофтЭксперт, г. Тула",2021-11-02 18:13:15+03:00,https://hh.ru/vacancy/14820007,"Программист, разработчик","CRM, ERP"
2,15518045,Программист графики,Санкт-Петербург,NaN,NaN,NaN,NaN,Открытая,NaN,NaN,От 1 года до 3 лет,Гибкий график,Полная занятость,314182,Saber Interactive,2021-11-02 12:14:47+03:00,https://hh.ru/vacancy/15518045,"Программист, разработчик","Программист, разработчик (other developer)"
5,16142474,Специалист по сопровождению ПО (техническая по...,Ижевск,20000.0,70000.0,RUR,0.0,Открытая,NaN,NaN,От 1 года до 3 лет,Полный день,Полная занятость,1020927,Центр Информационных Технологий БАРС,2021-10-18 09:02:39+03:00,https://hh.ru/vacancy/16142474,Специалист технической поддержки,Специалист технической поддержки
6,16487878,Программист С++,Санкт-Петербург,NaN,NaN,NaN,NaN,Открытая,NaN,NaN,От 1 года до 3 лет,Гибкий график,Полная занятость,314182,Saber Interactive,2021-11-02 12:14:48+03:00,https://hh.ru/vacancy/16487878,"Программист, разработчик","Программист, разработчик (C++)"
